In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from konlpy.tag import Okt
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
# CSV 파일 읽기
df = pd.read_csv("./레이블부여된공지사항(수정본).csv") 

notice_names = df['title'].tolist()

# 전처리 함수 정의
def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", text)

    # 숫자 정보 제거
    text = re.sub(r'\d+', '', text)

    # 형태소 분석
    tagger = Okt()
    words = tagger.morphs(text)

    # 불용어 제거
    stop_words = ['필독', '학기', '학년', '도', '년', '제', '회', '월', '학부', '일', '차', '년도', '안내']  # 불용어 리스트
    words = [word for word in words if word not in stop_words]

    # 분석된 형태소들을 공백으로 결합하여 문장으로 반환
    text = ' '.join(words)

    return text


preprocessed_notice_names = [preprocess_text(notice_name) for notice_name in notice_names]

In [4]:

# 토크나이저 생성
tokenizer = Tokenizer()

# 토크나이저가 데이터를 학습
tokenizer.fit_on_texts(preprocessed_notice_names)

# 데이터를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(preprocessed_notice_names)

# 가장 긴 문장의 길이를 확인
max_sequence_length = max(len(s) for s in sequences)
print('가장 긴 문장의 길이:', max_sequence_length)

# 어휘 사전의 크기를 확인 (+1을 하는 이유는 0인덱스를 고려하기 때문입니다.)
vocab_size = len(tokenizer.word_index) + 1
print('어휘 사전의 크기:', vocab_size)

# 모든 문장을 가장 긴 문장의 길이로 패딩 처리
padded_X = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

가장 긴 문장의 길이: 29
어휘 사전의 크기: 6093


In [5]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_notice_names)

tfidf_model = TfidfVectorizer().fit(preprocessed_notice_names)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['cluster'])

In [6]:
# 훈련 데이터 분리: train 데이터 80%, test 데이터 20%
X_train, X_test, y_train, y_test = train_test_split(padded_X, y, test_size=0.2, random_state=42)


In [7]:
# 타겟 변수 다중 클래스 원-핫 인코딩
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [9]:
# LSTM 모델 생성
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))

# 출력 뉴런의 개수와 활성화 함수를 클래스 개수에 따라 조정
num_classes = len(set(y))  # 클래스 개수 계산
if num_classes == 2:
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
else:
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
# 모델 훈련
model.fit(X_train, y_train_encoded, epochs=30, batch_size=32, validation_data=(X_test, y_test_encoded))

Epoch 1/30
248/248 [==============================] - 3s 13ms/step - loss: 0.1986 - accuracy: 0.9500 - val_loss: 4.6844 - val_accuracy: 0.6937
Epoch 2/30
248/248 [==============================] - 3s 13ms/step - loss: 0.1312 - accuracy: 0.9666 - val_loss: 4.5080 - val_accuracy: 0.7089
Epoch 3/30
248/248 [==============================] - 3s 13ms/step - loss: 0.0710 - accuracy: 0.9830 - val_loss: 4.6554 - val_accuracy: 0.7200
Epoch 4/30
248/248 [==============================] - 3s 12ms/step - loss: 0.0372 - accuracy: 0.9927 - val_loss: 4.6261 - val_accuracy: 0.7240
Epoch 5/30
248/248 [==============================] - 3s 12ms/step - loss: 0.0264 - accuracy: 0.9946 - val_loss: 4.5972 - val_accuracy: 0.7281
Epoch 6/30
248/248 [==============================] - 3s 12ms/step - loss: 0.0221 - accuracy: 0.9961 - val_loss: 4.6451 - val_accuracy: 0.7250
Epoch 7/30
248/248 [==============================] - 3s 12ms/step - loss: 0.0293 - accuracy: 0.9933 - val_loss: 4.7832 - val_accuracy: 0.7089

In [17]:
y_prob = model.predict(X_test, verbose=0) 
y_pred = y_prob.argmax(axis=-1)
accuracy = np.sum(y_pred.flatten() == y_test) / len(y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7371342078708375


In [25]:
# 새로운 입력 데이터 예측
new_text = input("새로운 문장을 입력하세요: ")
preprocessed_new_text = preprocess_text(new_text)
sequence = tokenizer.texts_to_sequences([preprocessed_new_text])
padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length, padding='post')
prediction = model.predict(padded_sequence)
predicted_label = label_encoder.inverse_transform([prediction.argmax()])[0]

# 예측 결과 출력
print(f"입력 문장: {new_text}")
print(f"예측된 레이블: {predicted_label}")

1/1 [==============================] - 0s 15ms/step
입력 문장: [전공] 2023학년도 1학기 전공신청 및 변경안내
예측된 레이블: 282
